# Importer les bibliothéques nécessaires et la base de données

In [ ]:
!pip install pvlib

In [ ]:
import pvlib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("dataavant.csv")
data.columns

## Vocabulaire :
* ghi : Irradiance globale horizontale
* dni : la composante directe
* dhi : la composante diffuse
* gti (global titled irradiance ) : Irradiance inclinée globale

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data = data.drop(columns='snow_depth')

In [ ]:
data = data.drop(columns='snow_water_equivalent')

In [ ]:
data = data.drop(columns='snow_soiling_rooftop')

In [ ]:
data = data.drop(columns='snow_soiling_ground')

In [ ]:
data = data.iloc[:-1]


In [ ]:
data.shape

# Détecter les valeurs Aberrantes
## Vérification suivantes sont effectuées :
* L’humidité relative (relative_humidity) sont entre 0 et 100 %
* Vérifier que wind_speed_10m, wind_speed_100m restent dans des valeurs réalistes (< 60 m/s en général)
* Vérifier que température de l'air (air_temp) est dans une plage réaliste selon la localisation.)

In [ ]:
data['relative_humidity'].describe()

In [ ]:
data['wind_speed_100m'].describe()

In [ ]:
data["wind_speed_10m"].describe()

In [ ]:
data['air_temp'].describe()

# Analyse des données
## Afficher les valeurs manquantes pour chaque

In [ ]:
data.isnull().sum()

## Feature engineering :
- Transformer l’heure en **sinusoïdes** (Hour_sin, Hour_cos).      
- Ajouter des indicateurs **saisonniers** (mois, jour de l’année).  

In [ ]:
# Extraire l'heure, le mois et le jour de l'année
data["period_end"] = pd.to_datetime(data["period_end"])
data["hour"] = data["period_end"].dt.hour
data["month"] = data["period_end"].dt.month
data["day_of_year"] = data["period_end"].dt.dayofyear

# Transformer l'heure en sinusoïdes
data["Hour_sin"] = np.sin(2 * np.pi * data["hour"] / 24)
data["Hour_cos"] = np.cos(2 * np.pi * data["hour"] / 24)

# Afficher les premières lignes pour vérifier
print(data[["period_end", "hour", "Hour_sin", "Hour_cos", "month", "day_of_year"]].head())

In [ ]:
# Ajouter une colonne avec une valeur fixe (ex: Latitude et Longitude)
data["latitude"] = 32.21047842880794  # Exemple : Paris
data["longitude"] = -7.938893140583626  # Exemple : Paris
data["altitude"] = 469.3   # Exemple : Altitude en mètres

# Ajouter une colonne vide (NaN)
data["tracker_tilt"] = None  # Colonne pour stocker l'angle d'inclinaison
data["tracker_azimuth"] = None  # Colonne pour l'azimut du tracker

# Vérifier les ajouts
print(df.head())

# Sauvegarder le fichier modifié (si nécessaire)
data.to_csv("data_updated.csv", index=False)

# Utiliser **PVLib** pour calculer les angles optimaux (azimuth, tilt).  
* NB : tilt (inclinaison optimale du panneau solaire)

In [ ]:
pip install pvlib pandas numpy matplotlib


In [ ]:
import pandas as pd
import numpy as np
import pvlib
import matplotlib.pyplot as plt

# 🔹 1. Charger la base de données
df = pd.read_csv("data_updated.csv")

# 🔹 2. Définir les coordonnées géographiques
latitude = df["latitude"].iloc[0]  # Latitude du site
longitude = df["longitude"].iloc[0]  # Longitude du site
altitude = df["altitude"].iloc[0]  # Altitude du site

# 🔹 3. Convertir les timestamps en format datetime et indexer
df["period_end"] = pd.to_datetime(df["period_end"])
df = df.set_index("period_end")

# 🔹 4. Calculer la position du soleil avec PVLib
solpos = pvlib.solarposition.get_solarposition(
    time=df.index,
    latitude=latitude,
    longitude=longitude,
    altitude=altitude,
    pressure=df["surface_pressure"] * 100,  # Conversion hPa -> Pa
    temperature=df["air_temp"]
)

# Ajouter les colonnes calculées
df["solar_zenith"] = solpos["zenith"]
df["solar_azimuth"] = solpos["azimuth"]

# 🔹 5. Appliquer les angles optimaux pour un tracker à double axes
df["tracker_tilt"] = 90 - df["solar_zenith"]  # Inclinaison perpendiculaire au soleil
df["tracker_azimuth"] = df["solar_azimuth"]  # Même azimut que le soleil

# 🔹 6. Filtrer les données où le soleil est sous l'horizon (éviter valeurs bizarres)
df = df[df["solar_zenith"] < 90]

# 🔹 7. Afficher un aperçu des résultats
print(df[["tracker_tilt", "tracker_azimuth"]].describe())

# 🔹 8. Visualiser les angles du tracker
fig, ax = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

ax[0].plot(df.index, df["tracker_tilt"], label="Inclinaison du tracker", color="blue")
ax[0].set_ylabel("Inclinaison (°)")
ax[0].set_title("Évolution des angles du tracker à double axes")
ax[0].grid()
ax[0].legend()

ax[1].plot(df.index, df["tracker_azimuth"], label="Azimut du tracker", color="green")
ax[1].set_ylabel("Azimut (°)")
ax[1].set_xlabel("Temps")
ax[1].grid()
ax[1].legend()

plt.show()


In [ ]:
df.to_csv("Solar_Tracking_OptimalesAngles.csv",index=False)

# Vérification des résultats de simulation

In [ ]:
import matplotlib.pyplot as plt

# Extraire l'heure de la journée
df["hour"] = df.index.hour

# Moyenne des angles pour chaque heure de la journée
hourly_avg = df.groupby("hour")[["tracker_tilt", "tracker_azimuth"]].mean()

# Tracer les courbes
fig, ax1 = plt.subplots(figsize=(10, 5))

ax1.plot(hourly_avg.index, hourly_avg["tracker_tilt"], label="Inclinaison (tilt)", color="blue", linewidth=2)
ax1.set_ylabel("Inclinaison (°)", color="blue")
ax1.tick_params(axis="y", labelcolor="blue")

ax2 = ax1.twinx()
ax2.plot(hourly_avg.index, hourly_avg["tracker_azimuth"], label="Azimut", color="green", linewidth=2)
ax2.set_ylabel("Azimut (°)", color="green")
ax2.tick_params(axis="y", labelcolor="green")

plt.title("Évolution des angles du tracker en fonction de l'heure")
ax1.set_xlabel("Heure de la journée")
ax1.set_xticks(range(0, 24, 2))  # Afficher une graduation toutes les 2 heures
plt.grid()

plt.show()
